<h1> 기본 웹 크롤러

In [115]:
import requests
import lxml.html
import re

In [217]:
response=requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root=lxml.html.fromstring(response.content)

for a in root.cssselect('.view_box a'):
    url=a.get('href')
    #print(url)

In [29]:
!pip install cssselect

In [215]:
root.make_links_absolute(response.url) #절대경로로 바꾸기

for a in root.cssselect('.view_box .book_tit a'): #javascript 없애기
    url=a.get('href')
    #print(url)

In [216]:
root.make_links_absolute(response.url) #절대경로로 바꾸기

for a in root.cssselect('.view_box .book_writer'): #javascript 없애기
    url=a.get('class')
    #print(url)

In [180]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [198]:
def scrape_detail_page(response):
    
    root=lxml.html.fromstring(response.content)
    ebook={
        'url':response.url,
        'title':root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price':root.cssselect('.pbr strong')[0].text_content(),
        'content':[normalize_spaces(p.text_content())
            for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
            if normalize_spaces(p.text_content())!='']
    }
    return ebook

In [199]:
def normalize_spaces(s):
    return re.sub(r'\s+','',s).strip()

In [204]:
def main():
    
    session=requests.Session()
    response=session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls=scrape_list_page(response)
    
    for url in urls:
        response=session.get(url)
        ebook=scrape_detail_page(response)
        print(ebook,"\n")


In [213]:
#if __name__=='__main__':
#    main()

<h1> 파이썬 크롤러 만들기

In [426]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os
import time

In [427]:
def db_save(NEWS_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            NEWS_LIST.to_sql(name='NEWS_LIST',con=con,index=False,if_exists='append')
        except Exception as e:
            print(str(e))
        print(len(NEWS_LIST), '건 저장완료')

In [ ]:
#cssselct 방법

def link(response):
    response=requests.get("https://news.daum.net/breakingnews/digital?page={}&regDate={}".format(PAGE,REG_DATE))
    root=lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)

    for a in root.cssselect('.box_etc .tit_thumb a'):
        url=a.get('href')
        print(url,a.text)

In [419]:
#xpath 방법

def link(response):
    response=requests.get("https://news.daum.net/breakingnews/digital?regDate={}".format(REG_DATE))
    root=lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)

    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a=li.xpath('div/strong/a')[0]
        url=a.get('href')
        print(url,a.text)

In [428]:
import re #정규식 사용 할려고
import string

def get_detail(url):
    body=[]
    punc='[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response=requests.get(url)
    root=lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'): #기사 내용, 첫줄
        if p.text:
            body.append(re.sub(punc,'',p.text)) #특수문자 제거
    
    full_body = ' '.join(body)
    
    return full_body
    #full_body=' '.join(body) #리스트 내용 하나로 합침 스페이스 공백으로 본문을 하나로 합치기 위해서
    
    #return body
    #return full_body

In [429]:
REG_DATE='20200819'
PAGE=50
MAX_PAGE=0

In [432]:
#뉴스 페이지 이동과 마지막 페이지 '>' 이거랑 숫자가 더이상 큰 것이 없다
while(True):
    df_list=[]
    response=requests.get('http://news.daum.net/breakingnews/digital?page={}&regDate={}'\
                            .format(page, REG_DATE))
    root=lxml.html.fromstring(response.content)
    
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a=li.xpath('div/strong/a')[0]
        url=a.get('href')
        article=get_detail(url)
        df = pd.DataFrame({'URL' : [url],'TITLE':[a.text],'ARTICLE' : [article]})
        df_list.append(df)  
            
    if df_list:   
        df_10 = pd.concat(df_list)
        db_save(df_10)
        
    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
        try:
            num=int(a.text)
            if max_page<num:
                max_page=num
        except:
            pass
        
    span=root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@class="btn_page btn_next"]')
    
    if (len(span)<=0)&(page>max_page):
        break
    else:
        page=page+1
    
    time.sleep(1)

<h1> 연습

In [370]:
#연습

def example(response):
    response=requests.get("https://www.justwatch.com/kr")
    root=lxml.html.fromstring(response.content)
    
    for li in root.xpath('//*[@id="base"]/div[1]/div/div[3]/ion-tab[3]/ion-content/div[4]/div/div/div[2]/div[1]'): #여기서부터 안먹음
        a=li.xpath('div/a')[0]
        url=a.get('href')
        print(url)

In [369]:
example(response)

In [352]:
# 연습

def ex(response):
    response=requests.get("https://book.naver.com/category/index.nhn?cate_code=280&list_type=&tab=new_book")
    root=lxml.html.fromstring(response.content)
    
    for li in root.xpath(' //*[@id="category_section"]/ol/li'):
        a=li.xpath('dl/dt/a')[0]
        url=a.get('href')
        print(url,a.text)
        

In [353]:
ex(response)

http://book.naver.com/bookdb/book_detail.nhn?bid=16656224 프로크리에이트 작법서
http://book.naver.com/bookdb/book_detail.nhn?bid=16655248 아이패드 드로잉 & 페인팅 with 프로크리에이트
http://book.naver.com/bookdb/book_detail.nhn?bid=16588285 꿈왕국과 잠자는 100명의 왕자님 공식 설정집
http://book.naver.com/bookdb/book_detail.nhn?bid=16655424 MATLAB
http://book.naver.com/bookdb/book_detail.nhn?bid=16624808 데이터 애널리틱스
http://book.naver.com/bookdb/book_detail.nhn?bid=16654764 한눈에 보이는 CMYK 배색&톤 가이드 - 아트지
http://book.naver.com/bookdb/book_detail.nhn?bid=16654765 한눈에 보이는 CMYK 배색&톤 가이드 - 모조지
http://book.naver.com/bookdb/book_detail.nhn?bid=16654766 한눈에 보이는 CMYK 배색&톤 가이드 - 아르떼지
http://book.naver.com/bookdb/book_detail.nhn?bid=16655323 JPA 퀵스타트
http://book.naver.com/bookdb/book_detail.nhn?bid=16627362 단계학습 한글 NEO 2016 기본 & 실무 프로젝트
http://book.naver.com/bookdb/book_detail.nhn?bid=16656198 ITQ 엑셀 2016
http://book.naver.com/bookdb/book_detail.nhn?bid=16627379 ITQ OA마스터 2016
http://book.naver.com/bookdb/book_detail.nhn?bid=16655348 JAVA
http://